### DB연결

In [7]:
import pandas as pd
from sqlalchemy import create_engine, text

# # 엑셀 파일 로딩
# file_path = "processed_output.xlsx"
# df = pd.read_excel(file_path)

# # 문자열 컬럼 강제 변환 (한글 오류 방지)
# for col in df.columns:
#     if df[col].dtype == object:
#         df[col] = df[col].astype(str)

# MariaDB 접속 정보
user = "root"
password = "1234"
host = "localhost"
port = 3306
database = "nav7"

# SQLAlchemy 연결
conn_str = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4"
engine = create_engine(conn_str)

def execute_sql(sql_query: str):
    with engine.connect() as conn:
        # text()로 감싸야 안전
        result = conn.execute(text(sql_query))
        # SELECT 쿼리면 결과를 fetch
        if sql_query.strip().lower().startswith("select"):
            rows = result.fetchall()
            # 컬럼명 가져오기
            columns = result.keys()
            # 결과 출력 또는 반환
            return columns, rows
        else:
            # SELECT 아닌 쿼리는 영향받은 row 수 반환
            return result.rowcount


# # 테이블 이름
# table_name = "project"


# # 테이블 생성 및 데이터 삽입
# df.head(0).to_sql(name=table_name, con=engine, if_exists='replace', index=False)
# df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

# print(f"✅ MariaDB에 '{table_name}' 테이블 생성 및 데이터 입력 완료.")


### OpenAI Key


In [ ]:
import openai
client = openai.OpenAI(api_key="")


### 개인정보 적용

In [8]:
from sqlalchemy import text

def fetch_project_by_id_sql(project_id: str):
    sql = f"SELECT * FROM project WHERE ID = '{project_id}'"
    try:
        result = execute_sql(sql)
        if isinstance(result, tuple):
            columns, rows = result
            if rows:
                return dict(zip(columns, rows[0]))  # 첫 번째 row만 dict로 변환
            else:
                return None
        else:
            return None
    except Exception as e:
        print(f"❌ Error fetching project by ID: {e}")
        return None


def get_refined_question(user_question: str, project_data: dict) -> str:
    project_info_str = "\n".join([
        f"{key}: {value}" for key, value in project_data.items() if value is not None
    ])

    prompt = f"""
    당신은 데이터베이스 및 커리어 분석 전문가입니다.
    아래는 한 사용자의 프로젝트 이력 정보와 자연어 질문입니다.

    이 사용자의 이력을 반영하여, 사용자의 질문을 더 구체적이고 상황에 맞게 '하나의 질문'으로 재작성해 주세요.
    불필요한 일반화 없이, 사용자의 실제 경력과 질문이 연결되도록 하세요.

    [프로젝트 이력 정보]
    {project_info_str}

    [원래 사용자 질문]
    "{user_question}"

    [이력 기반으로 재작성된 질문]
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 커리어 기반 질문 최적화 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=300
    )

    refined_question = response.choices[0].message.content.strip()
    return refined_question

In [12]:
#제일 마지막 row 기준
project_id = "EMP-100014"
user_question = "내경험과연차를고려할때, 최종적인CareerGoal은어떤게있을까? 나와비슷한연차와경험을가진다른사람들은어떤선택을했는지알려줘"

project_data = fetch_project_by_id_sql(project_id)
print(project_data)
if project_data:
    prompt = get_refined_question(user_question, project_data)
    print(prompt)
else:
    print("❌ 해당 ID의 프로젝트 정보를 찾을 수 없습니다.")


{'profileId': 52, 'ID': 'EMP-100014', 'StartYear': 1.0, 'EndYear': 5.0, 'Project': '반도체 설비관리 및 설비제어, SHE 통합시스템 구축', 'ProjectScale': 2.0, 'Roles': 'PL, PM', 'SkillSet1': '자동화 Eng..', 'SkillSet2': 'nan', 'SkillSet3': 'nan', 'SkillSet4': 'nan', 'Industry': '제조', 'CareerImpact': None, 'CareerImpactDesc': 'nan'}
"반도체 설비관리 및 설비제어, SHE 통합시스템 구축 프로젝트에서 PL 및 PM 역할을 수행한 5년 차 경력자로서, 제조 산업 분야에서의 커리어 목표를 설정하는 데 있어서 어떤 경로를 고려해야 할지 궁금합니다. 유사한 경력과 경험을 가진 다른 사람들은 어떤 선택을 하였는지, 그들의 경험을 통해 얻을 수 있는 가치에 대해 알려주시겠어요?"


### Text to SQL

In [17]:
from typing import List
def text_to_sql_with_intent_expansion(natural_language_query: str) -> List[str]:
    schema = """
    CREATE TABLE `project` (
      `ID` text DEFAULT NULL,
      `StartYear` double DEFAULT NULL,
      `EndYear` double DEFAULT NULL,
      `Project` text DEFAULT NULL,
      `ProjectScale` double DEFAULT NULL,
      `Roles` text DEFAULT NULL,
      `SkillSet1` text DEFAULT NULL,
      `SkillSet2` text DEFAULT NULL,
      `SkillSet3` text DEFAULT NULL,
      `SkillSet4` text DEFAULT NULL,
      `Industry` text DEFAULT NULL,
      `CareerImpact` double DEFAULT NULL
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """

    roles = """
    사용 가능한 Roles 값은 다음과 같습니다:
    Mainframe, System, Administrator, Application, Arch., Architect, Back-End, CICD, Cloud, Cost, DB2, DBA,
    Dev., Eng., Engineer, Engineering, Front-End, MPA, Mgmt, Middleware, Monitoring, NW, Net., Networking,
    Operation, Ops, PL, PM, PMO, Process, QA, SM, SaaS, Security, Server, Service, Solution,
    TA, Tech., Technical, Web, architect, controller, 교육, 마스터, 모듈개발, 운영, 인터페이스
    """

    skillsets = """
    사용 가능한 Skill Set 값은 다음과 같습니다:
    Front-end Dev, Back-end Dev, Mobile Dev, Factory 기획/설계, 자동화 Eng, 지능화 Eng,
    ERP_FCM, ERP_SCM, ERP_HCM, ERP_T&E, Biz. Solution, System/Network Eng,
    Middleware/Database Eng, Data Center Eng, Cyber Security,
    Application Architect, Data Architect, Technical Architect,
    Infra PM, Application PM, Solution PM, PMO, Quality Eng, Offshoring Service Professional,
    AI/Data Dev, Generative AI Dev, Generative AI Model Dev,
    Sales, Domain Expert, ESG/SHE, ERP, SCM, CRM, AIX,
    Strategy Planning, New Biz. Dev, Financial Mgmt, Human Resource Mgmt,
    Stakeholder Mgmt, Governance & Public Mgmt,
    Infra PM -- 대형PM, Application PM -- 대형PM, Solution PM -- 대형PM
    """

    # 1단계: 사용자 질문 기반으로 '확장된 여러 질문(의도)' 생성 요청
    intent_expand_prompt = f"""
    당신은 데이터베이스 및 업무 도메인 전문가입니다.
    아래 사용자의 질문에 대해,  
    관련되거나 확장 가능한 여러 개의 의미 있는 질문(의도)를 생성해 주세요.  
    가능한 경우 아래와 같이 리스트 형식으로 출력하세요(최대 3개):  
    - 질문 1  
    - 질문 2  
    - 질문 3  
    ...

    질문: "{natural_language_query}"
    """

    intent_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 자연어 이해 및 질문 확장 전문가입니다."},
            {"role": "user", "content": intent_expand_prompt}
        ],
        temperature=0,
        max_tokens=400
    )

    expanded_questions_text = intent_response.choices[0].message.content.strip()
    expanded_questions = [
        line.lstrip("- ").strip()
        for line in expanded_questions_text.splitlines()
        if line.startswith("- ")
    ]
    print(expanded_questions)
    sql_queries = []

    # 2단계: 확장된 질문 별로 SQL 생성
    for q in expanded_questions:
        sql_prompt = f"""
        아래는 데이터베이스 스키마입니다:
        {schema}

        Roles 목록:
        {roles}

        Skill Set 목록:
        {skillsets}

        위 내용을 참고하여, 다음 질문에 적합한 SQL 쿼리를 작성해 주세요.
        항상 'ID' 컬럼을 포함하여 조회하는 쿼리를 작성해 주세요.
        
        질문: {q}

        SQL 쿼리:
        """

        sql_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "당신은 SQL 전문가입니다."},
                {"role": "user", "content": sql_prompt}
            ],
            max_tokens=300,
            temperature=0,
        )

        sql_text = sql_response.choices[0].message.content.strip()
        if sql_text.startswith("```sql"):
            sql_text = sql_text.replace("```sql", "").replace("```", "").strip()
        sql_queries.append(sql_text)

    return sql_queries


In [18]:


sqls = text_to_sql_with_intent_expansion(prompt)
for i, sql in enumerate(sqls, 1):
    print(f"SQL {i}:\n{sql}\n")


['제조 산업 분야에서 커리어를 발전시키기 위해 어떤 교육 및 자격증이 유용할지 고려해 보셨나요?', '유사한 경력과 경험을 가진 전문가들이 산업 분야에서 성공을 거둔 비법이나 전략에 대해 알아보셨나요?', '제조 산업 분야에서의 커리어 목표를 설정할 때, 현재 시장 동향 및 미래 전망을 고려해 보셨나요?']
SQL 1:
SELECT ID, SkillSet1, SkillSet2, SkillSet3, SkillSet4
FROM project
WHERE Industry = 'Manufacturing'
AND (SkillSet1 IN ('Factory 기획/설계', 'ERP_FCM', 'ERP_SCM', 'ERP_HCM', 'ERP_T&E', 'Biz. Solution', 'System/Network Eng',
                   'Middleware/Database Eng', 'Data Center Eng', 'Cyber Security', 'Application Architect', 'Data Architect',
                   'Technical Architect', 'AI/Data Dev', 'Generative AI Dev', 'Generative AI Model Dev')
     OR SkillSet2 IN ('Factory 기획/설계', 'ERP_FCM', 'ERP_SCM', 'ERP_HCM', 'ERP_T&E', 'Biz. Solution', 'System/Network Eng',
                      'Middleware/Database Eng', 'Data Center Eng', 'Cyber Security', 'Application Architect', 'Data Architect',
                      'Technical Architect', 'AI/Data Dev', 'Generative AI Dev', 'Generative AI Model Dev')
     OR Sk

In [110]:
from collections import Counter

all_ids = []

for i, sql in enumerate(sqls, 1):
    print(f"\n▶ Executing SQL {i}:\n{sql}\n")
    try:
        result = execute_sql(sql)
        if isinstance(result, tuple):
            columns, rows = result
            print("✅ Query Result Columns:")
            print(columns)
            print("✅ Query Result Rows:")
            for row in rows:
                print(row)

            # columns를 리스트로 변환 후 ID 위치 찾기
            try:
                id_index = list(columns).index('ID')
            except ValueError:
                print("❌ 'ID' 컬럼이 결과에 없습니다.")
                continue

            # 이번 쿼리 결과에서 ID 리스트 추출해 all_ids에 누적
            all_ids.extend([row[id_index] for row in rows if row[id_index] is not None])

        else:
            print(f"✅ Affected rows: {result}")

    except Exception as e:
        print(f"❌ Error executing SQL {i}: {e}")

# 모든 쿼리 결과 통합 후 가장 많이 나온 ID 3개 출력
counter = Counter(all_ids)
most_common_ids = counter.most_common(3)

print("\n⭐ 전체 SQL 결과에서 조회된 ID 중 상위 3개:")
for id_val, count in most_common_ids:
    print(f"ID: {id_val}, Count: {count}")



▶ Executing SQL 1:
SELECT ID, Project, Roles, SkillSet1, SkillSet2, SkillSet3, SkillSet4, Industry, CareerImpact
FROM project
WHERE (Roles LIKE '%PL%' OR Roles LIKE '%PM%')
AND (SkillSet1 IN ('Factory 기획/설계', 'ERP_SCM', 'ERP_HCM', 'ERP_T&E', 'ESG/SHE'))
AND Industry = '반도체'
ORDER BY CareerImpact DESC;

✅ Query Result Columns:
RMKeyView(['ID', 'Project', 'Roles', 'SkillSet1', 'SkillSet2', 'SkillSet3', 'SkillSet4', 'Industry', 'CareerImpact'])
✅ Query Result Rows:

▶ Executing SQL 2:
SELECT ID, Industry, CareerImpact
FROM project
WHERE Roles IN ('Mainframe', 'System', 'Administrator', 'Application', 'Arch.', 'Architect', 'Back-End', 'CICD', 'Cloud', 'Cost', 'DB2', 'DBA',
                'Dev.', 'Eng.', 'Engineer', 'Engineering', 'Front-End', 'MPA', 'Mgmt', 'Middleware', 'Monitoring', 'NW', 'Net.', 'Networking',
                'Operation', 'Ops', 'PL', 'PM', 'PMO', 'Process', 'QA', 'SM', 'SaaS', 'Security', 'Server', 'Service', 'Solution',
                'TA', 'Tech.', 'Technical', 'We

In [ ]:
# 상위 3개 ID를 추출해 SQL IN 조건으로 사용
top_ids = [id_val for id_val, _ in most_common_ids]

# IN 절을 위한 문자열 생성
in_clause = ", ".join(f"'{id_val}'" for id_val in top_ids)

# SQL 쿼리 생성
query = f"""
SELECT *
FROM project
WHERE ID IN ({in_clause});
"""

# 실행
print("\n📌 상위 3개 ID에 해당하는 project 정보:")
try:
    result = execute_sql(query)
    if isinstance(result, tuple):
        columns, rows = result
        print("✅ Columns:")
        print(columns)
        print("✅ Rows:")
        for row in rows:
            print(row)
    else:
        print(f"✅ Affected rows: {result}")
except Exception as e:
    print(f"❌ Error fetching project details: {e}")



📌 상위 3개 ID에 해당하는 project 정보:
✅ Columns:
RMKeyView(['ID', 'StartYear', 'EndYear', 'Project', 'ProjectScale', 'Roles', 'SkillSet1', 'SkillSet2', 'SkillSet3', 'SkillSet4', 'Industry', 'CareerImpact', 'CareerImpactDesc'])
✅ Rows:
('EMP-525170', 1.0, 3.0, '대학교 학사시스템 구축', 2.0, '개발자', 'Front-end Dev.', 'Back-end Dev..', 'Application Architect', 'nan', '물류', 1.0, 'Tech 기반 엔지니어에서 분석설계개발 프로젝트에 참여함')
('EMP-525170', 4.0, 5.0, '항공사 성능 개선 프로젝트', 1.0, 'PM', 'Back-end Dev..', 'Application Architect', 'nan', 'nan', '물류', 0.0, 'nan')
('EMP-525170', 6.0, 7.0, '금융기관 운영', 2.0, '운영자', 'Middleware/Database Eng.', 'Application Architect', 'Technical Architect ', 'nan', '제2금융', 0.0, 'nan')
('EMP-525170', 7.0, 8.0, '자동차부품 SCM 구축', 2.0, 'Application, Architect', 'Application Architect', 'Middleware/Database Eng.', 'Back-end Dev..', 'nan', '제조', 0.0, 'nan')
('EMP-525170', 9.0, 10.0, '공공기관 시스템 리뉴얼', 2.0, 'DBA, 사업관리', 'Middleware/Database Eng.', 'Back-end Dev..', 'Application Architect', 'nan', '공공', 1.0, '공공 프로젝트

In [118]:
def rows_to_text(columns, rows) -> str:
    result = []
    for i, row in enumerate(rows, 1):
        row_dict = dict(zip(columns, row))
        row_str = "\n".join([f"{key}: {value}" for key, value in row_dict.items() if value is not None])
        result.append(f"🔹 프로젝트 {i}\n{row_str}")
    return "\n\n".join(result)

def generate_llm_response_from_projects(columns, rows, prompt: str) -> str:
    project_info_text = rows_to_text(columns, rows)

    full_prompt = f"""
    당신은 커리어 분석 전문가입니다.

    아래는 한 **조직 구성원**과 **질문자**의 프로젝트 및 경력 이력 정보입니다.  
    이 정보를 참고하여, 사용자 질문에 응답해 주세요.

    [1] 조직 구성원 이력 정보
    {project_info_text}

    [2] 질문자의 프로젝트 정보
    {project_data}

    아래 요청에 응답해 주세요:

    - 조직 구성원의 경력에서 중요한 역할(Role), 기술(SkillSet), 산업(Industry), 커리어 임팩트(CareerImpact) 등을 분석해 주세요.  
    - 질문자의 경력과 비교했을 때, 어떤 공통점 또는 차이점이 있는지 설명해 주세요.  
    - 사용자 질문에 대해, 구성원의 이력이 어떤 관점에서 유용하거나 참고할 만한지를 **구체적으로 설명**해 주세요.  
    - 어떤 프로젝트의 어떤 요소가 판단에 핵심적인 역할을 했는지 **명확히 근거를 제시**해 주세요.  
    - 마지막에, 질문자의 경력과 비교했을 때 조직 구성원의 이력에서 배울 수 있는 점이나 적용 가능한 인사이트를 종합적으로 제시하세요.

    [사용자 질문 또는 요청]
    {prompt}
    """.strip()


    response = client.chat.completions.create(
        model="gpt-4",  # 또는 "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "당신은 프로젝트 기반 커리어 분석 전문가입니다."},
            {"role": "user", "content": full_prompt.strip()}
        ],
        temperature=0.5,
        max_tokens=1500
    )

    return response.choices[0].message.content.strip()

In [119]:
llm_answer = generate_llm_response_from_projects(columns, rows, prompt)
print("\n🧠 LLM의 분석 결과:")
print(llm_answer)


🧠 LLM의 분석 결과:
조직 구성원들의 이력을 분석하면 다음과 같은 특징을 찾을 수 있습니다.

1. 역할: 구성원들은 다양한 역할을 수행하였으며, 그 중 대부분이 PL (프로젝트 리더) 또는 PM (프로젝트 매니저) 역할을 수행하였습니다. 이는 프로젝트의 전반적인 관리와 리더십 역할을 수행하였음을 의미합니다.

2. 기술: 구성원들은 각각의 프로젝트에서 다양한 기술을 사용하였습니다. 특히 'Domain Expert', 'System/Network Eng.', 'Quality Eng.', 'Infra PM' 등의 기술이 빈번하게 언급되었습니다. 이는 구성원들이 기술적인 역량 뿐만 아니라, 도메인 지식과 프로젝트 관리 능력을 갖추고 있음을 보여줍니다.

3. 산업: 구성원들은 제조, 금융, 공공 등 다양한 산업에서 프로젝트를 수행하였습니다. 이는 구성원들이 다양한 산업에 대한 이해도와 경험을 가지고 있음을 의미합니다.

4. 커리어 임팩트: 구성원들의 프로젝트 중 일부는 커리어에 중요한 영향을 미쳤습니다. 이는 특정 프로젝트가 구성원의 커리어 발전에 큰 역할을 하였음을 보여줍니다.

질문자의 이력을 보면, 질문자 역시 PL 및 PM 역할을 수행하였고, 제조 산업에서의 경험이 있으며, 특히 '반도체 설비관리 및 설비제어, SHE 통합시스템 구축' 프로젝트를 수행하였습니다. 이는 조직 구성원들의 이력과 많은 공통점을 가지고 있습니다.

이를 통해 질문자는 다음과 같은 커리어 목표를 고려해볼 수 있습니다.

1. 다양한 산업에 대한 이해도와 경험을 더욱 넓히는 것: 조직 구성원들은 다양한 산업에서의 프로젝트 경험을 통해 그 산업에 대한 깊은 이해도와 경험을 쌓았습니다. 질문자 역시 이러한 경험을 통해 다양한 산업에 대한 이해도를 높이고, 더 다양한 프로젝트를 수행할 수 있는 역량을 키울 수 있습니다.

2. 기술적인 역량과 도메인 지식을 더욱 강화하는 것: 조직 구성원들은 각각의 프로젝트에서 다양한 기술을 사용하였고, 특정 도메인에 대한 전문적인 지식을 가지고 있